In [13]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.xception import (
    Xception, preprocess_input, decode_predictions)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping

# Enable mixed precision training
mixed_precision.set_global_policy('mixed_float16')

# Efficient data loading
def load_and_preprocess_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

train_ds = tf.data.Dataset.list_files(train_dir + '/*/*')
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.list_files(val_dir + '/*/*')
val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)

# Build the model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # Ensure output layer is float32
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

# Evaluate the model
test_ds = tf.data.Dataset.list_files(test_dir + '/*/*')
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)

test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2f}")


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: ML_pneumonia_identification/Resources/train//*/*'

In [14]:
# detect image size and train?

In [15]:
# create a for loop to loop through all images?

In [16]:
#train model?

In [17]:
#test model?